In [1]:
import pandas
import os
import glob
import tqdm
import re
import numpy

In [2]:
metadata = "/data1/santiago/BBBC021/metadata/metadata.csv"
plates = set(map(lambda f: f[:-3], glob.glob("/data1/santiago/BBBC021/experiments/cnn/Week*")))
out_dir = "/data1/santiago/BBBC021/experiments/cnn/aggregate/"

In [3]:
def get_plate(plate):
    return os.path.basename(plate)[:-1]

In [4]:
metadf = pandas.read_csv(metadata)
metadf = metadf[metadf["Image_Metadata_Compound"] == "DMSO"]
wells = metadf["Metadata_Well"].unique()
meandf = pandas.DataFrame()
stddf = pandas.DataFrame()
for plate in tqdm.tqdm(plates):
    for well in wells:
        if get_plate(plate) in metadf["Metadata_Plate"].values and well in metadf["Metadata_Well"].values:
            platedf = pandas.DataFrame()
            filename = os.path.join(plate + well, "All.csv")
            if os.stat(filename).st_size > 1:
                welldf = pandas.read_csv(filename)
                welldf = welldf.select(lambda c: not('ImageNumber' in c or 'ObjectNumber' in c), axis=1)
                welldf = welldf.dropna()
                platedf = pandas.concat([platedf, welldf])
            else:
                print("Empty DataFrame:", filename)
    meanln = pandas.DataFrame([platedf.mean()])
    meanln["Plate"] = [get_plate(plate)]
    stdln = pandas.DataFrame([platedf.std()])
    stdln["Plate"] = [get_plate(plate)]
    meandf = pandas.concat([meandf, meanln])
    stddf = pandas.concat([stddf, stdln])
for column in stddf.select_dtypes([numpy.number]):
    if 0 in stddf[column].values:
        meandf = meandf.drop(column, axis=1)
        stddf = stddf.drop(column, axis=1)
meandf.to_csv(os.path.join(out_dir, "Mean.csv"), index=False)
stddf.to_csv(os.path.join(out_dir, "STD.csv"), index=False)
print("Done!")

100%|██████████| 55/55 [01:54<00:00,  2.08s/it]


Done!
